# Polar Express

Stefano Volpe #0000969766

University of Bologna

Introduction to Machine Learning

a.y. 2022/23

## Imports

In [1]:
import numpy as np
import tensorflow as tf

from keras.activations import elu, gelu, relu, sigmoid, softmax, softsign, \
  swish, tanh
from keras.callbacks import EarlyStopping
from keras.layers import Concatenate, Dense, Input, Normalization
from keras.losses import CategoricalCrossentropy
from keras.models import Model
from keras.optimizers import Nadam
from keras.utils import plot_model

## Generator

In [2]:
def polar_generator(batchsize, grid = (10, 10), noise = .002, flat = False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize, grid[0], grid[1]))
    xc = (x * grid[0]).astype(int)
    yc = (y * grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    # compute rho and theta and add some noise
    rho = np.sqrt(x ** 2 + y ** 2) + np.random.normal(scale = noise)
    theta = np.arctan(y / np.maximum(x, .00001)) + \
      np.random.normal(scale = noise)
    if flat:
      out = np.reshape(out, (batchsize, grid[0]*grid[1]))
    yield ((theta,rho),out)

## Dataset

The project requirements ask for a size of the validation greater or equal than 20000. In order for it to be one fourth of the training set (which is a good rule of thumb in general), 500000 was chosen. 

In [3]:
training_set_size, validation_set_size = 2000000, 500000

(training_theta, training_rho), training_maps = next(polar_generator(training_set_size, flat = True))
(validation_theta, validation_rho), validation_maps = next(polar_generator(training_set_size, flat = True))

## Metrics

The project requirements ask to compute the categorical accuracy of your model on your own, rather than using Keras's implementation.

In [4]:
def argmax_axis_1(input: tf.Tensor) -> int:
  return tf.argmax(input, axis = 1)

def my_categorical_accuracy(y_true : tf.Tensor, y_pred : tf.Tensor) -> tf.float64:
  # The right categories (according to our ground truth)
  y_true_argmax = argmax_axis_1(y_true)
  # The predictions our model assert with the most confidence
  y_pred_argmax = argmax_axis_1(y_pred)
  # Element-wise equality
  equalities = tf.equal(y_true_argmax, y_pred_argmax)
  # Since True converts to 1.0, accuracy and arithmetic mean are
  # equivalent
  equalities = tf.cast(equalities, tf.float64)
  return tf.reduce_mean(equalities)

## Model

In [7]:
normalization_set_size = 10000

def makeNetwork() -> Model:
  theta = Input(shape = (1, ), name = "Theta")
  theta_normalization = Normalization(axis = None, name = "ThetaNormalization")
  theta_normalization.adapt(training_theta[:normalization_set_size])
  theta_normalization = theta_normalization(theta)
  a1 = Dense(2, activation = softsign, name = "A1")(theta_normalization)
  a2 = Dense(4, activation = tanh, name = "A2")(a1)
  a3 = Dense(4, activation = sigmoid, name = "A3")(a2)

  rho = Input(shape = (1,), name = "Rho")
  rho_normalization = Normalization(axis = None, name = "RhoNormalization")
  rho_normalization.adapt(training_rho[:normalization_set_size])
  rho_normalization = rho_normalization(rho)
  b1 = Dense(4, activation = softsign, name = "B1")(rho_normalization)

  ab1 = Concatenate(name = "AB1")([a3, rho_normalization])
  ab2 = Dense(8, activation = swish, name = "AB2")(ab1)
  ab3 = Dense(8, activation = relu, name = "AB3")(ab2)
  ab4 = Dense(3, activation = gelu, name = "AB4")(ab3)
  out = Dense(100, activation = softmax, name = "out")(ab4)
  return Model([theta, rho], out)

polar_express = makeNetwork()
polar_express.build((None, 2))
polar_express.summary(show_trainable = False)
plot_model(
  polar_express,
  show_shapes = True,
  show_dtype = True,
  show_layer_activations = True,
)
polar_express.compile(
  Nadam(),
  CategoricalCrossentropy(),
  metrics = [my_categorical_accuracy]
)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Theta (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 ThetaNormalization (Normalizat  (None, 1)           3           ['Theta[0][0]']                  
 ion)                                                                                             
                                                                                                  
 A1 (Dense)                     (None, 2)            4           ['ThetaNormalization[0][0]']     
                                                                                                  
 A2 (Dense)                     (None, 4)            12          ['A1[0][0]']               

## Training and evaluation

Here is the training history. For each epoch, the network has been evaluated via
categorical accuracy on the validation set (see `val_my_categorical_accuracy`).

In [8]:
batch_size = 4096
epochs = 99
verbose = 2

polar_express.fit(
  (training_theta, training_rho),
  training_maps,
  batch_size,
  epochs,
  verbose,
  [EarlyStopping(monitor = 'val_loss', patience = 4)],
  validation_data = ((validation_theta, validation_rho), validation_maps)
)

Epoch 1/99
489/489 - 7s - loss: 3.8001 - my_categorical_accuracy: 0.0472 - val_loss: 2.8037 - val_my_categorical_accuracy: 0.1315 - 7s/epoch - 14ms/step
Epoch 2/99
489/489 - 5s - loss: 2.4514 - my_categorical_accuracy: 0.1849 - val_loss: 2.2448 - val_my_categorical_accuracy: 0.2215 - 5s/epoch - 11ms/step
Epoch 3/99
489/489 - 4s - loss: 2.0602 - my_categorical_accuracy: 0.2648 - val_loss: 1.8933 - val_my_categorical_accuracy: 0.2889 - 4s/epoch - 9ms/step
Epoch 4/99
489/489 - 4s - loss: 1.5978 - my_categorical_accuracy: 0.4106 - val_loss: 1.2499 - val_my_categorical_accuracy: 0.5257 - 4s/epoch - 9ms/step
Epoch 5/99
489/489 - 5s - loss: 0.9158 - my_categorical_accuracy: 0.7004 - val_loss: 0.7151 - val_my_categorical_accuracy: 0.7668 - 5s/epoch - 11ms/step
Epoch 6/99
489/489 - 4s - loss: 0.6010 - my_categorical_accuracy: 0.8273 - val_loss: 0.5331 - val_my_categorical_accuracy: 0.8389 - 4s/epoch - 9ms/step
Epoch 7/99
489/489 - 4s - loss: 0.4761 - my_categorical_accuracy: 0.8645 - val_loss: 